In [108]:
import os
import pandas as pd
import numpy  as np

In [42]:
import warnings
warnings.filterwarnings("ignore")

# Exploratory Data Analysis

The following notebook assumes:
- There hasn't been any problem getting the data.
- The data is located in the **data** repository, located at the same level than the **notebooks** repository that hosts this notebook 

In [37]:
DATA_REPOSITORY= '../data'

In [44]:
modeling_data_path= os.path.join(
    DATA_REPOSITORY,
    'PAKDD2010_Modeling_Data.txt'
)
variables_data_path= os.path.join(
    DATA_REPOSITORY,
    'PAKDD2010_VariablesList.XLS'
)

### Load the modeling data into a pd.DataFrame object

In [43]:
modeling_data= pd.read_csv(
    filepath_or_buffer= modeling_data_path, 
    encoding= "ISO-8859-1", 
    header= None, 
    delimiter='\t'
)

In [65]:
modeling_data.head()

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,1,C,5,Web,0,1,F,6,1,0,...,0,0,0,0,1,N,32,595,595,1
1,2,C,15,Carga,0,1,F,2,0,0,...,0,0,0,0,1,N,34,230,230,1
2,3,C,5,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,27,591,591,0
3,4,C,20,Web,0,1,F,2,0,0,...,0,0,0,0,1,N,61,545,545,0
4,5,C,10,Web,0,1,M,2,0,0,...,0,0,0,0,1,N,48,235,235,1


### Load the variables data into a pd.DataFrame object

In [63]:
df_variables= pd.read_excel(
    io=variables_data_path,  
    index_col= 0,
    header= 0
)
df_variables= df_variables\
    .set_index(df_variables.index - 1)

df_variables.index.name= 'COL_ID'

In [66]:
df_variables.head()

,Var_Title,Var_Description,Field_Content
COL_ID,,,
0,ID_CLIENT,Sequential number for the applicant (to be use...,"1-50000, 50001-70000, 70001-90000"
1,CLERK_TYPE,Not informed,C
2,PAYMENT_DAY,"Day of the month for bill payment, chosen by t...","1,5,10,15,20,25"
3,APPLICATION_SUBMISSION_TYPE,Indicates if the application was submitted via...,"Web, Carga"
4,QUANT_ADDITIONAL_CARDS,Quantity of additional cards asked for in the ...,"1,2,NULL"


### Rename modeling data columns

the **df_variables** dataframe has the title and description for the columns in **modeling_data** dataframe.

We need to:
- Check if variable titles are unique.
    - In case they aren't rename them in order to make sure there is a 1:1 Title - Column relationship
- Rename columns in **modeling_data** dataframe in order to clarify the dataset

In [79]:
pd.DataFrame(df_variables.Var_Title\
    .value_counts()\
    .sort_values(ascending= False)[df_variables.Var_Title\
        .value_counts()\
        .sort_values(ascending= False) > 1
    ]
)

,Var_Title
EDUCATION_LEVEL,2


There are two columns with the **EDUCATION_LEVEL** title

Lets understand each column with the variables' descriptions

In [96]:
ed_level_col_descrip_df = pd.DataFrame(
    df_variables[
        df_variables.Var_Title == "EDUCATION_LEVEL"
    ].Var_Description
)

In [97]:
for i, row in ed_level_col_descrip_df.iterrows():
    print(
        f"--- Column ID: {i:2d} ---",
        f"{row.Var_Description}",
        sep='\n'
    )

--- Column ID:  9 ---
Edducational level in gradual order not informed
--- Column ID: 43 ---
Mate's educational level in gradual order not informed


#### As we can see, the COL_ID 9 refers to the clients educational level and COL_ID 43 refers to the client mate's educational level. We will then:

- Create a dictionary to rename **modeling_data** dataframe columns to map the Column ID with the Variable Title
- Customly, define the COL_ID 09 as CLI_EDUCATION_LEVEL
- Customly, define the COL_ID 43 as MATE_EDUCATION_LEVEL 

#### Finally, we will rename the columns using the created dictionary as a mapper and set the ID_CLIENT as the dataframe index


In [102]:
rename_dict= {column: df_variables.loc[column].Var_Title for column in df_variables.index}
rename_dict[9]  = f"CLI_{rename_dict[9]}" 
rename_dict[43] = f"MATE_{rename_dict[43]}" 

In [106]:
modeling_data = modeling_data.rename(
    columns= rename_dict
).set_index('ID_CLIENT')

With the raw data structured, we will save it with the name defined in **RAW_DATA_FILENAME** as a csv

In [118]:
RAW_DATA_FILENAME= 'raw_modeling_data.csv'

In [119]:
RAW_DATA_PATH= os.path.join(
    DATA_REPOSITORY,
    RAW_DATA_FILENAME
)

In [120]:
RAW_DATA_PATH

'../data/raw_modeling_data.csv'

In [121]:
modeling_data.to_csv(
    path_or_buf= RAW_DATA_PATH
)

## Feature Engineering 

First thing we will do is to check:
- The data type of each column
- The number of unique values for each column
- The number of missing values for each column
- The percentage of missing values over the total number of records in the **modeling_data** dataframe

In [113]:
col_dypes = modeling_data.dtypes
col_dypes.name= "Data type"
col_n_unique_values= data.nunique()
col_n_unique_values.name= "# unique values"
col_n_null_values= data.isna().sum()
col_n_null_values.name= "# NULL Values"
col_per_null_values= np.round(data.isna().sum() / data.shape[0] * 100, 2)
col_per_null_values.name= "% NULL Values"

In [116]:
columns_metrics= pd.concat(
    objs=[
        col_dypes, 
        col_n_unique_values, 
        col_n_null_values, 
        col_per_null_values
    ], 
    axis= 1
)

In [117]:
columns_metrics

,Data type,# unique values,# NULL Values,% NULL Values
CLERK_TYPE,object,1,0,0.00
PAYMENT_DAY,int64,6,0,0.00
APPLICATION_SUBMISSION_TYPE,object,3,0,0.00
QUANT_ADDITIONAL_CARDS,int64,1,0,0.00
POSTAL_ADDRESS_TYPE,int64,2,0,0.00
SEX,object,4,0,0.00
MARITAL_STATUS,int64,8,0,0.00
QUANT_DEPENDANTS,int64,17,0,0.00
CLI_EDUCATION_LEVEL,int64,1,0,0.00
STATE_OF_BIRTH,object,29,0,0.00


In [27]:
mask_drop_col_condition_01 = ((columns_metrics["# unique values"] == 1) & (columns_metrics["# NULL Values"] == 0))a
mask_drop_col_condition_02 = (columns_metrics["% NULL Values"] > 60)

In [28]:
columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02]

,# unique values,# NULL Values,% NULL Values
CLERK_TYPE,1,0,0.00
QUANT_ADDITIONAL_CARDS,1,0,0.00
EDUCATION_LEVEL,1,0,0.00
FLAG_MOBILE_PHONE,1,0,0.00
PROFESSIONAL_CITY,2236,33783,67.57
PROFESSIONAL_BOROUGH,5057,33783,67.57
MATE_EDUCATION_LEVEL,6,32338,64.68
FLAG_HOME_ADDRESS_DOCUMENT,1,0,0.00
FLAG_RG,1,0,0.00
FLAG_CPF,1,0,0.00


In [29]:
len(columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02].index)

12

In [32]:
col_to_keep = [
    col for col in data.columns if col not in columns_metrics[mask_drop_col_condition_01 | mask_drop_col_condition_02].index
]

In [35]:
len(data[col_to_keep].columns)

41

In [33]:
data[col_to_keep].corr()[data.columns[-1]]

/var/folders/7m/ryt4krsd3q58y7d6y0f2_k1w0000gp/T/ipykernel_4807/2558802318.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[col_to_keep].corr()[data.columns[-1]]


PAYMENT_DAY                       0.070633
POSTAL_ADDRESS_TYPE              -0.002423
MARITAL_STATUS                   -0.030380
QUANT_DEPENDANTS                  0.012157
NACIONALITY                      -0.000952
RESIDENCE_TYPE                    0.019168
MONTHS_IN_RESIDENCE              -0.029336
FLAG_EMAIL                       -0.008067
PERSONAL_MONTHLY_INCOME           0.002819
OTHER_INCOMES                     0.004442
FLAG_VISA                        -0.005398
FLAG_MASTERCARD                  -0.017658
FLAG_DINERS                       0.002240
FLAG_AMERICAN_EXPRESS             0.000337
FLAG_OTHER_CARDS                  0.001410
QUANT_BANKING_ACCOUNTS            0.011997
QUANT_SPECIAL_BANKING_ACCOUNTS    0.011997
PERSONAL_ASSETS_VALUE            -0.003886
QUANT_CARS                        0.011706
MONTHS_IN_THE_JOB                -0.007073
PROFESSION_CODE                  -0.011593
OCCUPATION_TYPE                   0.040216
MATE_PROFESSION_CODE             -0.034505
PRODUCT    